In [4]:
# 计算简单算术移动平均线MA 
import pandas as pd
def MA(data,N1,N2,N3):
   MAN1=data['Clsprc'].rolling(N1).mean()
   MAN2=data['Clsprc'].rolling(N2).mean()
   MAN3=data['Clsprc'].rolling(N3).mean()
   return (MAN1,MAN2,MAN3)

# 计算指数平滑移动平均线EMA
def my_MACD(data, com):
    alpha = 1 / (1 + com)
    y = (1 - alpha) * y + alpha * x

def MACD(data): 
    import numpy as np 
    # EMA12 = pd.ewma(data['Clsprc'].values, 12)
    # EMA26 = pd.ewma(data['Clsprc'].values, 26)
    EMA12 = pd.DataFrame.ewm(data['Clsprc'], span=12).mean()
    EMA26 = pd.DataFrame.ewm(data['Clsprc'], span=26).mean()
    # pd.ewma(stock_data['close'], span=ma)
    # pd.DataFrame.ewm(stock_data['close'], span=ma).mean()
    DIF=EMA12- EMA26
    DIF = np.array(DIF)
    DEA=np.zeros((len(DIF)))
    MACD=np.zeros((len(DIF)))
    for t in range(len(DIF)):
        if t==0:
             DEA[t]= DIF[t]
        if t>0:
             DEA[t]=(2*DIF[t]+8*DEA[t-1])/10
        MACD[t]=2*(DIF[t]-DEA[t])
    return MACD

#计算随机指标KDJ
def KDJ(data,N):
    import numpy as np 
    # Lmin=pd.rolling_min(data['Loprc'].values,N)
    # Lmax=pd.rolling_max(data['Hiprc'].values,N)
    Lmax=data['Loprc'].rolling(N).min()
    Lmin=data['Hiprc'].rolling(N).max()
    # df_test['col_name'].rolling(ma).mean()
    # df_test['col_name'].rolling(ma).mean()

    RSV=(data['Clsprc'].values-Lmin)/(Lmax-Lmin)
    RSV = np.array(RSV)
    K=np.zeros((len(RSV)))
    D=np.zeros((len(RSV)))
    J=np.zeros((len(RSV)))
    for t in range(N,len(data)):
        if t==0:
            K[t]=RSV[t]
            D[t]=RSV[t]
        if t>0:
            K[t]=2/3*K[t-1]+1/3*RSV[t]
            D[t]=2/3*D[t-1]+1/3*K[t]
        J[t]=3*D[t]-2*K[t]
    return (K,D,J)

#计算相对强弱指标RSI
def RSI(data,N):
    import numpy as np
    z=np.zeros(len(data)-1) 
    z[data.iloc[1:,2].values-data.iloc[0:-1,2].values>=0]=1
    z[data.iloc[1:,2].values-data.iloc[0:-1,2].values<0]=-1
    # z1=pd.rolling_sum(z==1,N)
    # z2=pd.rolling_sum(z==-1,N)
    z1=pd.DataFrame(z==1).rolling(N).sum()
    z2=pd.DataFrame(z==-1).rolling(N).sum()
    z1 = np.array(z1)
    z2 = np.array(z2)
    # df_test['col_name'].rolling(ma).mean()
    # df_test['col_name'].rolling(ma).mean()
    rsi=np.zeros((len(data)))
    for t in range(N-1,len(data)-1):
        rsi[t]=z1[t]/(z1[t]+z2[t])
    return rsi

def BIAS(data,N):
    import numpy as np
    bias=np.zeros((len(data)))
    # man=pd.rolling_mean(data.iloc[:,2].values,N)
    man = pd.DataFrame(data.iloc[:,2].values).rolling(N).mean()
    man = np.array(man)
    for t in range(N-1,len(data)):
        bias[t]=(data.iloc[t,2]-man[t])/man[t]
    return bias

def OBV(data):
    import numpy as np
    obv=np.zeros((len(data)))
    for t in range(len(data)):
        if t==0:
            obv[t]=data['Dnshrtrd'].values[t]
        if t>0:
            if data['Clsprc'].values[t]>=data['Clsprc'].values[t-1]:
                obv[t]=obv[t-1]+data['Dnshrtrd'].values[t]
            if data['Clsprc'].values[t]<data['Clsprc'].values[t-1]:
                obv[t]=obv[t-1]-data['Dnshrtrd'].values[t]
    return obv

def cla(data):
    import numpy as np
    y=np.zeros(len(data)) 
    z=np.zeros(len(y)-1)
    for i in range(len(z)):
        z[data.iloc[1:,2].values-data.iloc[0:-1,2].values>0]=1
        z[data.iloc[1:,2].values-data.iloc[0:-1,2].values==0]=0
        z[data.iloc[1:,2].values-data.iloc[0:-1,2].values<0]=-1
        y[i]=z[i]  
    return y

In [6]:
import pandas as pd
data=pd.read_excel('data.xlsx')
MA= MA(data,5,10,20) 
macd=MACD(data)
kdj=KDJ(data,9)
rsi6=RSI(data,6)
rsi12=RSI(data,12)
rsi24=RSI(data,24)
bias5=BIAS(data,5)
bias10=BIAS(data,10)
bias20=BIAS(data,20)
obv=OBV(data)
y=cla(data)

#将计算出的技术指标与交易日期以及股价的涨跌趋势利用字典整合在一起
pm={'交易日期':data['Trddt'].values}
PM=pd.DataFrame(pm)
DF={'MA5':MA[0],'MA10':MA[1],'MA20':MA[2],'MACD':macd,
    'K':kdj[0],'D':kdj[1],'J':kdj[2],'RSI6':rsi6,'RSI12':rsi12,
    'RSI24':rsi24,'BIAS5':bias5,'BIAS10':bias10,'BIAS20':bias20,'OBV':obv}
DF=pd.DataFrame(DF)
s1=PM.join(DF)

y1={'涨跌趋势':y}
ZZ=pd.DataFrame(y1)
s2=s1.join(ZZ)

#去掉空值
ss=s2.dropna()
#将ss中第6列不为0的值提取出来，存放到Data中
Data=ss[ss.iloc[:,6].values!=0]

#提取训练和预测数据
x1=Data['交易日期']>='2017-01-01'
x2=Data['交易日期']<='2017-11-30'
xx=x1&x2
index=xx.values==True
index1=xx.values==False
x_train=Data.iloc[index,1:15]
y_train=Data.iloc[index,[15]]
x_test=Data.iloc[index1,1:15]
y_test=Data.iloc[index1,[15]]


#数据标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)


from sklearn.linear_model import LogisticRegression as LR
clf = LR()
clf.fit(x_train, y_train)
result=clf.predict(x_test)
sc=clf.score(x_train, y_train)
'''
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=8, random_state=1)
     # 多个隐含层hidden_layer_sizes(5,2)
clf.fit(x_train, y_train) 
result=clf.predict(x_test)
sc=clf.score(x_train, y_train)

from sklearn import svm
clf = svm.SVC()
clf.fit(x_train, y_train) 
result=clf.predict(x_test)
sc=clf.score(x_train, y_train)
'''
result=pd.DataFrame(result)
#提取预测样本的交易日期
ff=Data.iloc[index1,0]
#将预测结果与实践结果整合在一起，进行比较
pm1={'交易日期':ff.values,'预测结果':result.iloc[:,0].values,'实际结果':y_test.iloc[:,0].values}
result1=pd.DataFrame(pm1)
z=result1['预测结果'].values-result1['实际结果'].values
R=len(z[z==0])/len(z)

r_list=[]
r_trd=[]
for t in range(len(result1)-1):
    if result1['预测结果'].values[t]==1:
        p1=data.loc[data['Trddt'].values== result1['交易日期'].values[t],'Clsprc'].values
        dt=data.loc[data['Trddt'].values>result1['交易日期'].values[t],['Trddt','Clsprc']]
        dt=dt.sort_values('Trddt')
        p2=dt['Clsprc'].values[0]
        r=(p2-p1)/p1
        r_list.append(r)
        r_trd.append(result1['交易日期'].values[t])
r_total=sum(r_list)
trd_r={'交易日期':r_trd,'收益率':r_list}
trd_r=pd.DataFrame(trd_r)
print(trd_r)



          交易日期                       收益率
0   2017-12-01   [0.0019404915912030634]
1   2017-12-04    [0.028405422853453807]
2   2017-12-15   [0.0028929604628736695]
3   2017-12-18   [0.0051282051282051325]
4   2017-12-19     [0.00478316326530619]
5   2017-12-20    [0.003173595683909802]
6   2017-12-21  [-0.0006327111673520903]
7   2017-12-22    [0.009496676163342852]
8   2017-12-25   [-0.005644402634054553]
9   2017-12-27   [0.0012836970474967633]
10  2017-12-28    [0.026923076923076918]


C:\Users\16405\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
